# Solving Sudoku with (M)ILP using PYOMO
As an example of integer programming and use of PYOMO, we will solve Sudoku. Warning: It might take the fun out of Sudoku...

As a challenge, we will try to solve the [world's hardest Sudoku](https://www.vg.no/nyheter/utenriks/i/agwQM/er-dette-verdens-vanskeligste-sudoku):  

![The world's hardest Sudoku](sudoku.jpg)

Note that there are many codes out there for solving Sudoku, also without using integer programming. For example [this](https://norvig.com/sudoku.html) by Peter Norvig.

We start by setting up PYOMO to solve a 'Concrete' model (which will be this specific Sudoku)

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# create model
m = ConcreteModel()

For each cell, we use a binary for each possible number (1 to 9). Therefore, we have three natural index sets (which in this case are the same, so we could make do with only one). In addition, we need index sets for each 9x9 square.

In [2]:
# index sets
m.value = RangeSet(1,9)             # Possible values
m.row = RangeSet(1,9)               # Row values
m.column = RangeSet(1,9)            # Column values
m.squaresVertical = RangeSet(1,3)   # 3 squares vertically
m.squaresHorizontal = RangeSet(1,3) # 3 squares horizontally

Then we define our variables. For each cell, a binary for each possible number. That is, 9x9x9 = 729 binaries in all.

In [3]:
# Variable
m.x = Var(m.value, m.row, m.column, domain=Binary)

Then we add constraints. 
In each cell, only one of the binaries can be one (there can only be a single number in that cell):

In [4]:
# Constraints
m.oneValue = ConstraintList()
for r in m.row:
    for c in m.column:
        m.oneValue.add( sum(m.x[v,r,c] for v in m.value) == 1 )

In each row, a value can only be assigned one cell:

In [5]:
m.valueRow = ConstraintList()
for v in m.value:
    for c in m.column:
        m.valueRow.add( expr = sum(m.x[v,r,c] for r in m.row) == 1 )

The same for each column:

In [6]:
m.valueColumn = ConstraintList()
for v in m.value:
    for r in m.row:
        m.valueColumn.add( expr = sum(m.x[v,r,c] for c in m.column) == 1 )

Then, for each of the nine squares, each value can only be used once:

In [7]:
m.valueSquares = ConstraintList()
for v in m.value:
    for p in m.squaresVertical:
        for q in m.squaresHorizontal:
            m.valueSquares.add( m.x[v,3*p-2,3*q-2] + m.x[v,3*p-2,3*q-1] + m.x[v,3*p-2,3*q] \
                + m.x[v,3*p-1,3*q-2] + m.x[v,3*p-1,3*q-1] + m.x[v,3*p-1,3*q] \
                    + m.x[v,3*p,3*q-2] + m.x[v,3*p,3*q-1] + m.x[v,3*p,3*q] == 1 )

Then we insert the given values of our particular Sudoku (the world's hardest), column by column:

In [8]:
m.given = ConstraintList()
m.given.add( m.x[8,2,1] == 1 )
m.given.add( m.x[4,4,1] == 1 )

m.given.add( m.x[7,3,2] == 1 )
m.given.add( m.x[1,5,2] == 1 )
m.given.add( m.x[6,7,2] == 1 )

m.given.add( m.x[5,1,3] == 1 )
m.given.add( m.x[3,6,3] == 1 )
m.given.add( m.x[4,8,3] == 1 )

m.given.add( m.x[3,1,4] == 1 )
m.given.add( m.x[2,6,4] == 1 )
m.given.add( m.x[5,7,4] == 1 )

m.given.add( m.x[1,3,5] == 1 )
m.given.add( m.x[7,5,5] == 1 )

m.given.add( m.x[5,4,6] == 1 )
m.given.add( m.x[9,9,6] == 1 )

m.given.add( m.x[5,3,7] == 1 )
m.given.add( m.x[3,4,7] == 1 )
m.given.add( m.x[7,9,7] == 1 )

m.given.add( m.x[2,2,8] == 1 )
m.given.add( m.x[8,6,8] == 1 )
m.given.add( m.x[3,8,8] == 1 )

m.given.add( m.x[6,5,9] == 1 )
m.given.add( m.x[9,7,9] == 1 )

The above are all the constraints. We also need an objective function. This is not important here, since we only want to find a solution (the only one). Randomly, we choose the objective to be the first binary.

In [9]:
# Objective
m.OBJ = Objective(expr = m.x[1,1,1])

We want to use the open source solver glpk as solver, and solve:

In [10]:
opt = SolverFactory('glpk')
results = opt.solve(m)

We can have a look at the solver output. The number of constraints are given by the loops above: 9x9 + 9x9 + 9x9 + 9x3x3 = 324. Why the number of variables are 730 rather than 729, I don't know.

In [11]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 348
  Number of variables: 730
  Number of nonzeros: 2940
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 35
      Number of created subproblems: 35
  Error rc: 0
  Time: 0.04587864875793457
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

Pyomo copies the result of the optimization into the defined variables (unless we specify otherwise). We can therefore look at the solution by displaying the variables. Pretty printing this as a Sudoku board is left as an exercise :)

In [34]:
m.x.display()

x : Size=729, Index=x_index
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1, 1) :     0 :   1.0 :     1 : False : False : Binary
    (1, 1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 3) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 4) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 5) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 6) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 7) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 8) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1, 9) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 3) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 4) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 5) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 6) :     0 :   0